In [1]:
import os
import torch
import numpy as np
from pgd import PGD
from dknn import AdvDKNN, DKNN
from models.vgg import VGG16
from torchvision.datasets import CIFAR10

In [2]:
root = "./datasets"
model_weights = "./model_weights/cifar_vgg_at.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
download = not os.path.exists(os.path.join(root,"cifar-10-python.tar.gz"))
random_seed = 1234
class_samp_size = 2000

trainset = CIFAR10(root=root, train=True, download=download)
testset = CIFAR10(root=root, train=False, download=download)

np.random.seed(random_seed)
subset = [
    np.random.choice(
        np.where(np.array(trainset.targets)==i)[0],
        size=class_samp_size,
        replace=False
    ) for i in range(10)
]
train_data = torch.cat([
    torch.FloatTensor(trainset.data[inds].transpose(0,3,1,2)/255) for inds in subset
], dim=0)
train_targets = torch.arange(10).repeat_interleave(class_samp_size)

model = VGG16()
model.load_state_dict(torch.load(model_weights, map_location=device))
model.eval()
model.to(device)

VGG16(
  (f1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (f2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (f3

In [3]:
pgd = PGD(eps=8 / 255., step_size=2 / 255., max_iter=10, random_init=True, batch_size=128)

x, y = (
    torch.FloatTensor(testset.data.transpose(0, 3, 1, 2) / 255.)[:1000],
    torch.LongTensor(testset.targets)[:1000]
)

x_adv = pgd.generate(model, x, y, device=device)

In [4]:
dknn = DKNN(
    model,
    train_data,
    train_targets,
    hidden_layers=[3,],
    device=device
)
advdknn = AdvDKNN(
    model,
    train_data,
    train_targets,
    hidden_layers=[3,],
    device=device,
    eps=8 / 255.,
    step_size=2 / 255.,
    max_iter=10,
    random_init=True
)

pred_benign_dknn = dknn(x.to(device)).argmax(axis=1)
acc_benign_dknn = (pred_benign_dknn == y.numpy()).astype("float").mean()
print(f"The benign accuracy by DKNN is {acc_benign_dknn}")

pred_adv_dknn = dknn(x_adv.to(device)).argmax(axis=1)
acc_adv_dknn = (pred_adv_dknn == y.numpy()).astype("float").mean()
print(f"The adversarial accuracy by DKNN is {acc_adv_dknn}")

pred_benign_advdknn = advdknn(x.to(device)).argmax(axis=1)
acc_benign_advdknn = (pred_benign_advdknn == y.numpy()).astype("float").mean()
print(f"The benign accuracy by Adv-DKNN is {acc_benign_advdknn}")

pred_adv_advdknn = advdknn(x_adv.to(device)).argmax(axis=1)
acc_adv_advdknn = (pred_adv_advdknn == y.numpy()).astype("float").mean()
print(f"The adversarial accuracy by Adv-DKNN is {acc_adv_advdknn}")

100%|██████████| 1/1 [00:08<00:00,  8.96s/it]


The benign accuracy by DKNN is 0.82
The adversarial accuracy by DKNN is 0.433
The benign accuracy by Adv-DKNN is 0.816
The adversarial accuracy by Adv-DKNN is 0.459


In [5]:
advdknn = AdvDKNN(
    model,
    train_data,
    train_targets,
    hidden_layers=[3,],
    device=device,
    eps=4 / 255.,
    step_size=2 / 255.,
    max_iter=10,
    random_init=True
)
pred_benign_advdknn = advdknn(x.to(device)).argmax(axis=1)
acc_benign_advdknn = (pred_benign_advdknn == y.numpy()).astype("float").mean()
print(f"The benign accuracy by Adv-DKNN is {acc_benign_advdknn}")

pred_adv_advdknn = advdknn(x_adv.to(device)).argmax(axis=1)
acc_adv_advdknn = (pred_adv_advdknn == y.numpy()).astype("float").mean()
print(f"The adversarial accuracy by Adv-DKNN is {acc_adv_advdknn}")

100%|██████████| 1/1 [00:08<00:00,  8.67s/it]


The benign accuracy by Adv-DKNN is 0.819
The adversarial accuracy by Adv-DKNN is 0.449


In [6]:
dknn = DKNN(
    model,
    train_data,
    train_targets,
    hidden_layers=[2,3],
    device=device
)
pred_benign_dknn = dknn(x.to(device)).argmax(axis=1)
acc_benign_dknn = (pred_benign_dknn == y.numpy()).astype("float").mean()
print(f"The benign accuracy by DKNN is {acc_benign_dknn}")

pred_adv_dknn = dknn(x_adv.to(device)).argmax(axis=1)
acc_adv_dknn = (pred_adv_dknn == y.numpy()).astype("float").mean()
print(f"The adversarial accuracy by DKNN is {acc_adv_dknn}")

100%|██████████| 2/2 [00:27<00:00, 13.80s/it]


The benign accuracy by DKNN is 0.817
The adversarial accuracy by DKNN is 0.478


In [7]:
advdknn = AdvDKNN(
    model,
    train_data,
    train_targets,
    hidden_layers=[2,3],
    device=device,
    eps=8 / 255.,
    step_size=2 / 255.,
    max_iter=10,
    random_init=True
)
pred_benign_advdknn = advdknn(x.to(device)).argmax(axis=1)
acc_benign_advdknn = (pred_benign_advdknn == y.numpy()).astype("float").mean()
print(f"The benign accuracy by Adv-DKNN is {acc_benign_advdknn}")

pred_adv_advdknn = advdknn(x_adv.to(device)).argmax(axis=1)
acc_adv_advdknn = (pred_adv_advdknn == y.numpy()).astype("float").mean()
print(f"The adversarial accuracy by Adv-DKNN is {acc_adv_advdknn}")

100%|██████████| 2/2 [00:27<00:00, 13.60s/it]


The benign accuracy by Adv-DKNN is 0.799
The adversarial accuracy by Adv-DKNN is 0.525


In [9]:
advdknn = AdvDKNN(
    model,
    train_data,
    train_targets,
    hidden_layers=[3,],
    weight=[0,0],
    device=device,
    eps=8 / 255.,
    step_size=2 / 255.,
    max_iter=10,
    random_init=True
)
pred_benign_advdknn = advdknn(x.to(device)).argmax(axis=1)
acc_benign_advdknn = (pred_benign_advdknn == y.numpy()).astype("float").mean()
print(f"The benign accuracy by Adv-DKNN is {acc_benign_advdknn}")

pred_adv_advdknn = advdknn(x_adv.to(device)).argmax(axis=1)
acc_adv_advdknn = (pred_adv_advdknn == y.numpy()).astype("float").mean()
print(f"The adversarial accuracy by Adv-DKNN is {acc_adv_advdknn}")

100%|██████████| 1/1 [00:08<00:00,  8.70s/it]


The benign accuracy by Adv-DKNN is 0.699
The adversarial accuracy by Adv-DKNN is 0.535


In [ ]:
advdknn = AdvDKNN(
    model,
    train_data,
    train_targets,
    hidden_layers=[3,],
    weight=0,
    device=device,
    eps=4 / 255.,
    step_size=2 / 255.,
    max_iter=10,
    random_init=True
)
pred_benign_advdknn = advdknn(x.to(device)).argmax(axis=1)
acc_benign_advdknn = (pred_benign_advdknn == y.numpy()).astype("float").mean()
print(f"The benign accuracy by Adv-DKNN is {acc_benign_advdknn}")

pred_adv_advdknn = advdknn(x_adv.to(device)).argmax(axis=1)
acc_adv_advdknn = (pred_adv_advdknn == y.numpy()).astype("float").mean()
print(f"The adversarial accuracy by Adv-DKNN is {acc_adv_advdknn}")